In [1]:
!pip --quiet install pytorch-warmup
%cd /mnt/workspace/shell/

#这个cd就是我家里主机上就这样


/mnt/workspace/shell


In [2]:
from IPython.display import clear_output 
!pip --quiet install pytorch_spiking pytorch_lightning #pytorch_forecasting 
clear_output()

In [3]:
import os
import time
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import pickle

import torch
import torch.nn as nn
from torch.nn.modules import TransformerEncoder, TransformerEncoderLayer, LayerNorm
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import pytorch_spiking
import pytorch_warmup as warmup

import matplotlib.pyplot as plt

In [4]:
import torch.backends.cudnn as cudnn

from models_s4.s4.s4 import S4Block as S4  # Can use full version instead of minimal S4D standalone below
from models_s4.s4.s4d import S4D

CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.


In [5]:
config = {
  "plots": {
        "show_plots": False,
        "xticks_interval": 1200,
        "color_actual": "#001f3f",
        "color_train": "#3D9970",
        "color_val": "#0074D9",
        "color_test": "#FF4136",
        "color_pred_train": "#3D9970",
        "color_pred_val": "#0074D9",
        "color_pred_test": "#FF4136",
    },
    "data": {
        "train_split_size": 0.80,
        "input_window": 30,
        "output_window": 10,
        "train_batch_size": 3,
        "eval_batch_size": 1,
        "scaler": "normal"
    }, 
    "model_transformer": {
        "feature_size": 250,
        "nhead": 10,
        "num_layers": 2,
        "dropout": 0.2,
        "out_features": 1,
        "init_range": 2, #0.5
        "lr": 0.0002, #0.0001,
        "loss": "dilate"
    },
    "paths": {
        "drive": {
            "agg_trade": {
                "train": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/",
                "test": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/", 
            },
            "orderbook": {
                "train": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/",
                "test": "/content/drive/MyDrive/IP/Repos/HFTransformer/input_data/",
            },
            "models": "/content/drive/MyDrive/IP/Repos/HFTransformer/models/",
            "figures": "/content/drive/MyDrive/IP/Repos/HFTransformer/figures/",
            "utils": "/content/drive/MyDrive/IP/Repos/HFTransformer/utils/",
        },
        
        "local": {
            "agg_trade": {
                "train": "./data/input_data/",
                "test": "./data/input_data/", 
            },
            "orderbook": {
                "train": "./data/input_data/",
                "test": "./data/input_data/",
            },
            "models": "./models/",
            "figures": "./figures/",
        }
    }
}

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
drive = False
print(device)

cuda


In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

## Data preparation: augmenting raw financial data

In [8]:
def augment_trade_data(df, lag, forecast_window=None):
    '''
    Augmenting input data.
    '''
    if forecast_window:
        df['lag_return'] = np.log(df['price'].shift(forecast_window)/df['price'].shift(forecast_window+1))
        return df.iloc[forecast_window+1:,:]
    if lag == 0:
        return df
    else:
        col_name = 'log_lag'+str(lag)+'_price'
        df[col_name] = np.log(df.price) - np.log(df.price).shift(lag)
        return df.iloc[lag:,:]
    
#后续会用到 别急 就是模拟了一个正常交易的时候的延迟

## Defining Transformer Model

In [9]:
if tuple(map(int, torch.__version__.split('.')[:2])) == (1, 11):
    print("WARNING: Dropout is bugged in PyTorch 1.11. Results may be worse.")
    dropout_fn = nn.Dropout
if tuple(map(int, torch.__version__.split('.')[:2])) >= (1, 12):
    dropout_fn = nn.Dropout1d
else:
    dropout_fn = nn.Dropout2d


class S4Model(nn.Module):

    def __init__(
        self,
        d_input,
        d_output=10,
        d_model=256,
        n_layers=4,
        dropout=0.2,
        prenorm=False,
    ):
        super().__init__()
        self.prenorm = prenorm
        self.encoder = nn.Linear(d_input, d_model)
        # Stack S4 layers as residual blocks
        self.s4_layers = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        for _ in range(n_layers):
            self.s4_layers.append(
                S4D(d_model, dropout=dropout, transposed=True, lr=min(0.001, 0.01))
            )
            self.norms.append(nn.LayerNorm(d_model))
            self.dropouts.append(dropout_fn(dropout))
        # Linear decoder
        self.decoder = nn.Linear(d_model, d_model)

    def forward(self, x):
        """
        Input x is shape (B, L, d_input)
        """
        x = self.encoder(x)  # (B, L, d_input) -> (B, L, d_model)
        if torch.isnan(x).any():
            print("NaN detected in 10101010 output")
        x = x.transpose(-1, -2)  # (B, L, d_model) -> (B, d_model, L)
        for layer, norm, dropout in zip(self.s4_layers, self.norms, self.dropouts):
            # Each iteration of this loop will map (B, d_model, L) -> (B, d_model, L)

            z = x
            if self.prenorm:
                # Prenorm
                z = norm(z.transpose(-1, -2)).transpose(-1, -2)

            # Apply S4 block: we ignore the state input and output
            z, _ = layer(z)

            # Dropout on the output of the S4 block
            z = dropout(z)

            # Residual connection
            x = z + x

            if not self.prenorm:
                # Postnorm
                x = norm(x.transpose(-1, -2)).transpose(-1, -2)

        x = x.transpose(-1, -2)
        if torch.isnan(x).any():
            print("NaN detected in -1 output")
        x = self.decoder(x)  # (B, d_model) -> (B, d_output)
        if torch.isnan(x).any():
            print("NaN detected in 0 output")
        return x
# torch.Size([256, 64, 100])
# torch.Size([100, 256, 36])

In [10]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, num_layers):
        super(TransformerEncoderLayer, self).__init__()
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead), 
            num_layers=num_layers
        )
    def forward(self, x):
        return self.transformer_encoder(x)


class S4Transformer(nn.Module):
    def __init__(self, input_size, d_model=36, nhead=4, num_layers=3, output_size=1):
        super(S4Transformer, self).__init__()
        self.s4 = S4Model(
                d_input=38,#len(features),
                d_output=1,
                d_model=36,
                n_layers=3,
                dropout=0.1,
                prenorm="store_true")
        self.transformer_encoder = TransformerEncoderLayer(d_model, nhead, num_layers)
        self.fc_out = nn.Linear(d_model, output_size)

    def forward(self, x):
        # Input x shape: [batch_size, seq_len, input_size]
        if torch.isnan(x).any():
            print("NaN detected in 1 output")
        x = self.s4(x)
        if torch.isnan(x).any():
            print("NaN detected in 2 output")
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)
        if torch.isnan(x).any():
            print("NaN detected in 3 output")
        x = x.permute(1, 0, 2)
        if torch.isnan(x).any():
            print("NaN detected in 4 output")

        x = x.mean(dim=1)
        if torch.isnan(x).any():
            print("NaN detected in 5 output")
        x = self.fc_out(x)
        if torch.isnan(x).any():
            print("NaN detected in 6 output")
        
        return x

## Defining Loader

In [11]:
class TimeSeriesDataset(Dataset):
    '''
    Class for converting LOB data into model inputs.
    '''
    def __init__(self, x, y):
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])


def prepare_data_x(data, window_size, lag):
    '''
    Windows the input data for the ML models.
    '''
    n_row = data.shape[0] - window_size + 1
    subset = data[:window_size]
    subset_mean = np.mean(subset, axis=0)
    output = np.zeros([n_row, window_size, len(subset_mean)])
    x_mean = np.zeros([n_row, len(subset_mean)])
    x_std = np.zeros([n_row, len(subset_mean)])
    for idx in range(n_row):
        subset = data[idx:idx+window_size]
        subset_mean = np.mean(subset, axis=0)
        subset_std = np.std(subset, axis=0) + 0.01
        subset_norm = (subset-subset_mean)/subset_std
        x_mean[idx,:] = subset_mean
        x_std[idx,:] = subset_std
        output[idx,:,:] = subset_norm
    x_mean = np.array(x_mean)
    x_std = np.array(x_std)
    return output[:-lag-1], output[-1], x_mean, x_std


def prepare_data_y(x, window_size, lag):
    '''
    Windows the target data for the ML models.
    '''
    output = np.zeros([len(x)-window_size-lag])
    std = 1.1*np.sqrt(lag)+lag*0.01
    for idx in range(0,len(x)-window_size-lag):
        output[idx] = np.log(x[window_size+lag-1+idx,0]/x[window_size-1+idx,0])*10_000
    output = output/std
    return output


def prepare_data(normalized_prices_train, dates_train, normalized_prices_test, dates_test, config, lag=1, plot=False):
    '''
    Returns input and target data.
    '''
    data_x, data_x_unseen, x_mean, x_std = prepare_data_x(normalized_prices_train, window_size=100, lag=lag)
    data_y = prepare_data_y(normalized_prices_train, window_size=100, lag=lag)
    split_index = int(data_y.shape[0]*0.8)
    data_x_train = data_x[:split_index]
    data_x_val = data_x[split_index:]
    data_y_train = data_y[:split_index]
    data_y_val = data_y[split_index:]

    return split_index, data_x_train, data_y_train, data_x_val, data_y_val


#这些就是来帮助把数据分成batch的

## Defining Custom Losses

In [12]:
def quantile_loss(y, y_pred, quantile):
  '''
  Computes quantile loss
  Standard quantile loss as defined in the "Training Procedure" section of
  the main TFT paper
  '''
  if quantile < 0 or quantile > 1:
    raise ValueError(
        'Illegal quantile value={}! Values should be between 0 and 1.'.format(
            quantile))

  prediction_underflow = y - y_pred
  q_loss = quantile * torch.max(prediction_underflow, torch.zeros_like(prediction_underflow)) + (
      1. - quantile) * torch.max(-prediction_underflow, torch.zeros_like(prediction_underflow))

  return torch.sum(q_loss, axis=-1)


#quantile loss在让模型有自信值很有帮助

## Defining Helper Functions

In [13]:
criterion_dict = {"MAE":nn.L1Loss, "MSE":nn.MSELoss, "QuantileLoss":quantile_loss}

def compute_loss(labels, output, src, criterion):
    '''
    Computes loss
    '''
    if isinstance(output, torch.Tensor):
        if len(labels.shape) != len(output.shape):
            if len(labels.shape) > 1:
                if labels.shape[1] == output.shape[1]:
                    labels = labels.unsqueeze(2)
                else:
                    labels = labels.unsqueeze(0)
    loss = 0
    loss = criterion(output, labels.float())
    return loss


def train_step(model, opt, criterion, data_loader, takes_target, device,
                       num_targets=1, forward_params={}):
    '''
    Performs training of a single model. Runs through one epoch of the data.
    '''
    i = 0
    running_loss = 0.0
    model.train()
    for src, trg in data_loader:
        opt.zero_grad()
        if takes_target:
            forward_params["t"] = trg.to(device)
        src = src.to(device)
        trg = trg.to(device)
        
        # Ensure all tensors in forward_params are on the correct device
        # print(src.shape)
        output = model(src,**forward_params)
        output = output.squeeze()
        if num_targets == 1:
            labels = trg
        elif num_targets > 1:
            labels = trg[:, :, 0:num_targets]

        
        loss = compute_loss(labels, output, src, criterion[0])
        loss.backward()
        opt.step()
        running_loss += loss.item()
        i += 1
    total_loss = running_loss
    return total_loss


def validation(val_loader, model, criterion, device, num_targets=1):
    '''
    Computes the validation loss metrics.
    '''
    crit_losses = dict.fromkeys(criterion, 0)
    model.eval()
    labels = torch.Tensor(0).to(device)
    labels_all = torch.Tensor(0).to(device)
    output_all = torch.Tensor(0).to(device)
    with torch.no_grad():
        for src, targ in val_loader:
            output = torch.Tensor(0).to(device)
            src = src if isinstance(src, list) else src.to(device)
            targ = targ if isinstance(targ, list) else targ.to(device)
            output = model(src.float())
            output = output.squeeze()
            output_all = torch.cat((output_all, output))
            if num_targets == 1:
                labels = targ
            elif num_targets > 1:
                labels = targ[:, :, 0:num_targets]
            for crit in criterion:
                loss = compute_loss(labels, output, src, crit)
                crit_losses[crit] += loss.item()
            labels_all = torch.cat((labels_all, labels))
    return list(crit_losses.values())[0], output_all, labels_all
def forecast(data_loader, model, criterion, forecast_horizon, device, num_targets=1):
    '''
    Forecasting
    '''
    crit_losses = dict.fromkeys(criterion, 0)
    model.eval()
    output_decoder = torch.Tensor(0).to(device)
    labels = torch.Tensor(0).to(device)
    labels_all = torch.Tensor(0).to(device)
    counter = 0
    with torch.no_grad():
        for src, targ in data_loader:
            if (counter % forecast_horizon) == 0:
                src = src if isinstance(src, list) else src.to(device)
                targ = targ if isinstance(targ, list) else targ.to(device)
                output = model(src.float())
                #output = output.reshape(1,-1)
                output_decoder = torch.cat((output_decoder, output))
                if num_targets == 1:
                    labels = targ
                elif num_targets > 1:
                    labels = targ[:, :, 0:num_targets]
                for crit in criterion:
                    loss = compute_loss(labels, output, src, crit)
                    crit_losses[crit] += loss.item()
                labels_all = torch.cat((labels_all, labels))
            counter += 1
    return list(crit_losses.values())[0], output_decoder, labels_all


#这些属于常规操作了

## Defining Trainer

In [14]:
def strategy_evaluator(true, pred):
    '''
    Evaluates strategy regarding correct buys and sells
    '''
    total_buys, total_sells, total_holds = np.sum(true>0), np.sum(true<0), np.sum(true==0)
    total_correct_buys, total_correct_sells, total_correct_holds = 0, 0, 0
    for idx in range(len(true)):
        for jdx in range(len(true[0])):
            if true[idx,jdx] > 0 and pred[idx,jdx] > 0:
                total_correct_buys += 1
            elif true[idx,jdx] < 0 and pred[idx,jdx] < 0:
                total_correct_sells += 1
            elif true[idx,jdx] == 0 and pred[idx,jdx] == 0:
                total_correct_holds += 1
    total_correct_buys_r, total_correct_sells_r, total_correct_holds_r = (total_correct_buys/total_buys),(total_correct_sells/total_sells),(total_correct_holds/total_holds)
    return total_correct_buys_r.round(3), total_correct_sells_r.round(3), total_correct_holds_r.round(3)

In [15]:
def trainer(model, train_loader, validation_loader, test_loader, criterion, opt, scheduler,
            warmup_scheduler, max_epochs, batch_size, forecast_horizon, takes_target, shuffle=False,
            num_targets=1, plot_prediction=True, save_path='/mnt/workspace/shell/results_Transencwithlineardec'
, LAG=0):
    '''
    Training method
    '''
    start_time = time.time()
    
    data_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=False, sampler=None, batch_sampler=None, num_workers=10)
    validation_data_loader = DataLoader(validation_loader, batch_size=batch_size, shuffle=False, sampler=None, batch_sampler=None, num_workers=10)
    test_data_loader = DataLoader(test_loader, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=2)
    forecast_data_loader = DataLoader(validation_loader, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=2)
    
    for epoch in range(1, max_epochs+1):

        total_loss = train_step(model, opt, criterion, data_loader, takes_target, device, num_targets=num_targets)
        val_loss = 0
        if plot_prediction:
            val_loss, val_values, true_values = forecast(forecast_data_loader, model, criterion, forecast_horizon=forecast_horizon,
                                                                   device=device, num_targets=num_targets)
            fig, ax = plt.subplots(1, 1, figsize = (18, 8))
            ax.plot(true_values.cpu().view(-1), label='truth', alpha=0.3)
            ax.plot(val_values.cpu().view(-1), label='forecast', alpha=0.8)
            ax.set_xlim(left=0, right=len(true_values.cpu().view(-1)))
            plt.show()
        else:
            val_loss, val_values, true_values = validation(validation_data_loader, model, criterion, device,
                                                            num_targets=num_targets)
            # val_loss, val_values, true_values, src_all
        preds, trues = val_values.cpu().numpy(), true_values.cpu().numpy()#, src_all.cpu().numpy()


        # print(f'preds {preds.shape}')
        # print(f'trues {trues.shape}')

        results = 0
    
            
        r2_sklearn = r2_score(trues, preds)

        elapsed = time.time() - start_time
        print('-' * 88)
        print('| epoch {:3d} | {:5.2f} s | train loss {:5.5f} | val loss {:5.5f} | lr {:1.8f} | r2 sklearn: {:1.5f} | b, s, h: {:}|'.format(
                        epoch, elapsed, total_loss, val_loss, scheduler.get_last_lr()[0], r2_sklearn, results))
        print('-' * 88)
        start_time = time.time()

        if save_path:
            results = {
                    'model': 'Transencwithlineardec',
                    'pred_len': forecast_horizon,
                    'epoch': epoch,
                    'train_loss': total_loss,
                    'val_loss': val_loss,
                    'r2_val_sklearn': r2_sklearn            
            }

            df = pd.DataFrame([results])
            df.to_csv(os.path.join(save_path, 'results.csv'), mode='a', header=not os.path.exists(save_path), index=False)
            save_directory = os.path.join(save_path, "Transencwithlineardec")
            if not os.path.exists(save_directory):
                os.makedirs(save_directory)
            if r2_sklearn >0.02 :
                torch.save(model.state_dict(), os.path.join(save_path,"Transencwithlineardec",f'_epoch_{epoch}_time_{time.time()}_r2_{r2_sklearn}.pt'))

        with warmup_scheduler.dampening():
            scheduler.step()


## Model and Training

## Optimal paramater search

In [16]:
# date_train = 'all' 
# date_test = 'all'
date_train = 'All_to_Sept'
date_test = 'All_to_Sept'


# drive = None
# if False:
#     if drive:
#         agg_trade = pd.read_csv(config["paths"]["drive"]["agg_trade"]["train"]+date_train+'/orderbook.csv')    
#         sys.path.append(config["paths"]["drive"]["utils"])
#     else: 
#         agg_trade = pd.read_csv(config["paths"]["local"]["agg_trade"]["train"]+date_train+'/orderbook_agg_trade_dollarvol_drop_duplicate_price.csv')
#         agg_trade_test = pd.read_csv(config["paths"]["local"]["agg_trade"]["test"]+date_test+'/orderbook_agg_trade_dollarvol_drop_duplicate_price.csv')
idx = 0
agg_trade = pd.read_csv(config["paths"]["local"]["agg_trade"]["train"]+date_train+'/orderbook_agg_trade_dollarvol.csv')
# agg_trade_test = pd.read_csv(config["paths"]["local"]["agg_trade"]["test"]+date_test+'/orderbook_agg_trade_dollarvol.csv')
agg_trade['w_midprice'] = (agg_trade['ask1']*agg_trade['askqty1']+agg_trade['bid1']*agg_trade['bidqty1'])/(agg_trade['askqty1']+agg_trade['bidqty1'])
# total_rows = agg_trade.shape[0]
# total_vols = agg_trade.shape[1]
# #agg_trade['price'] = agg_trade['w_midprice']
# # agg_trade_test = agg_trade[4_500_000:]
# print(total_rows,total_vols)
# orderbook = augment_trade_data(agg_trade, lag=0, forecast_window=100)
# features = ['price', 'lag_return',
#                 'bid1', 'bidqty1', 'bid2', 'bidqty2', 'bid3', 'bidqty3', 'bid4', 'bidqty4', 'bid5', 'bidqty5',
#                 'bid6', 'bidqty6', 'bid7', 'bidqty7', 'bid8', 'bidqty8', 'bid9', 'bidqty9',
#                 'ask1', 'askqty1', 'ask2', 'askqty2', 'ask3', 'askqty3', 'ask4', 'askqty4', 'ask5', 'askqty5',
#                 'ask6', 'askqty6', 'ask7', 'askqty7', 'ask8', 'askqty8', 'ask9', 'askqty9']
        
# data_array =  np.array(orderbook[features][1_000_000:1_720_000])
# data_array =  np.array(orderbook[features][:])
# # 查看 shape
# print(data_array.shape)
# # 打印前 10 行（索引 0 到 9）
# print("前 10 行:")
# print(data_array[0:10])

# # 打印索引从 1,000,000 到 1,000,009 的数据
# # 注意：确保 data_array 的长度足够大，否则可能会引发索引错误
# print("\n索引从 1,000,000 到 1,000,009 的数据:")
# print(data_array[719_990:720_000])  

In [17]:



model_name = 'Transwiths4_tmp_1'

save_path = os.path.join(f'/mnt/workspace/shell/tmp/{model_name}/training_details/HFTransformer/results_HFformer',
                            str(int(time.time()))+'_results.csv')

# save_path = f'/mnt/workspace/shell/tmp/{model_name}/training_details/HFTransformer/results_HFformer'
# # save_path=None
# save_path = f'/home/gaen/Documents/codespace-gaen/Simons/{model_name}/training_details/HFTransformer/results_HFformer'
filepath = f'/mnt/workspace/shell/tmp/{model_name}/training_details/HFTransformer/results_HFformer/HFformer'
# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)
os.makedirs(filepath, exist_ok=True)

forecast_history = 100
epochs = 32
batch_size = 256 #64 for linear decoder

forecast_windows = [i for i in range(1,32)]

for forecast_window in forecast_windows:
    
    # orderbook = augment_trade_data(agg_trade, lag=0, forecast_window=forecast_window)
    orderbook = augment_trade_data(agg_trade, lag=0, forecast_window=forecast_window)
    # total_orw = orderbook.shape[0]
    # print(total_orw)
    features = ['price', 'lag_return',
                'bid1', 'bidqty1', 'bid2', 'bidqty2', 'bid3', 'bidqty3', 'bid4', 'bidqty4', 'bid5', 'bidqty5',
                'bid6', 'bidqty6', 'bid7', 'bidqty7', 'bid8', 'bidqty8', 'bid9', 'bidqty9',
                'ask1', 'askqty1', 'ask2', 'askqty2', 'ask3', 'askqty3', 'ask4', 'askqty4', 'ask5', 'askqty5',
                'ask6', 'askqty6', 'ask7', 'askqty7', 'ask8', 'askqty8', 'ask9', 'askqty9']


    split_index, data_x_train, data_y_train, data_x_val, data_y_val = prepare_data(np.array(orderbook[features][1_000_000:1_350_000]),
                                                                                                                            np.array(agg_trade.datetime[899_999:1_000_000]),
                                                                                                                            np.array(orderbook[features][60_000:60_600]),
                                                                                                                            np.array(agg_trade.datetime[60_000:60_600]),
                                                                                                                            config, lag=forecast_window, plot=False)

    # data_x_train_shape = data_x_train.shape[1]
    # data_y_train_shape = data_y_train.shape[1]
    # data_x_val_shape = data_x_val.shape[1]
    # data_y_val_shape = data_y_val.shape[1]
    # print( data_x_train_shape , data_y_train_shape ,data_x_val_shape, data_y_val_shape)
    train_loader = TimeSeriesDataset(data_x_train, data_y_train)
    val_loader = TimeSeriesDataset(data_x_val, data_y_val)
    test_loader = None

    model_custom = S4Transformer(input_size=len(features), d_model=36,
                 num_layers=3, output_size=1).to(device)

    criterion = nn.MSELoss(reduction='sum')
    optimizer = optim.AdamW(model_custom.parameters(), lr=0.1, amsgrad=True)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
    warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period=1000)

    trainer(model_custom, train_loader, val_loader, test_loader, [criterion], optimizer, scheduler, warmup_scheduler, epochs, batch_size=batch_size,
        forecast_horizon=forecast_window, takes_target=False, plot_prediction=False, save_path=save_path, LAG=forecast_window)
    
    del data_x_train 
    del data_y_train
    del data_x_val
    del data_y_val

    torch.save(model_custom, f'/mnt/workspace/shell/tmp/{model_name}/transformer_enclinear_forecasting_FINAL_horizon_{forecast_window}.pt')
    print(f'Done with prediction len {forecast_window}.')


/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


----------------------------------------------------------------------------------------
| epoch   1 | 57.49 s | train loss 34862.81595 | val loss 1407.57908 | lr 0.10000000 | r2 sklearn: 0.00755 | b, s, h: 0|
----------------------------------------------------------------------------------------


/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


----------------------------------------------------------------------------------------
| epoch   2 | 56.17 s | train loss 34611.48947 | val loss 1364.51249 | lr 0.09800000 | r2 sklearn: 0.03792 | b, s, h: 0|
----------------------------------------------------------------------------------------


/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

----------------------------------------------------------------------------------------
| epoch  30 | 58.05 s | train loss 48548.40816 | val loss 6742.86166 | lr 0.05566167 | r2 sklearn: 0.11875 | b, s, h: 0|
----------------------------------------------------------------------------------------


/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


----------------------------------------------------------------------------------------
| epoch  31 | 58.28 s | train loss 48262.10545 | val loss 6742.82724 | lr 0.05454843 | r2 sklearn: 0.11876 | b, s, h: 0|
----------------------------------------------------------------------------------------


/mnt/software/anaconda3/envs/python3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


----------------------------------------------------------------------------------------
| epoch  32 | 58.20 s | train loss 48751.22987 | val loss 6786.29190 | lr 0.05345746 | r2 sklearn: 0.11308 | b, s, h: 0|
----------------------------------------------------------------------------------------
Done with prediction len 31.


In [ ]:
if False:
    date_train = 'all' 
    date_test = 'all'
    drive = None
    if drive:
        agg_trade = pd.read_csv(config["paths"]["drive"]["agg_trade"]["train"]+date_train+'/orderbook.csv')    
        sys.path.append(config["paths"]["drive"]["utils"])
    else:
        agg_trade = pd.read_csv(config["paths"]["local"]["agg_trade"]["train"]+date_train+'/orderbook_agg_trade_dollarvol_drop_near_duplicate_price.csv')
        agg_trade_test = pd.read_csv(config["paths"]["local"]["agg_trade"]["test"]+date_test+'/orderbook_agg_trade_dollarvol_drop_near_duplicate_price.csv')
    idx = 0
    agg_trade['w_midprice'] = (agg_trade['ask1']*agg_trade['askqty1']+agg_trade['bid1']*agg_trade['bidqty1'])/(agg_trade['askqty1']+agg_trade['bidqty1'])

    #agg_trade['price'] = agg_trade['w_midprice']
    agg_trade_test = agg_trade[4_500_000:]

In [41]:
if False:
    save_path = os.path.join('./home/gaen/Documents/codespace-gaen/Simons/models_drop_duplicate_near/training_details/HFTransformer/results_HFformer',
                                str(int(time.time()))+'_results.csv')

    save_path = '/home/gaen/Documents/codespace-gaen/Simons/models_drop_duplicate_near/training_details/HFTransformer/results_HFformer'
    # save_path=None

    forecast_history = 100 
    epochs = 16
    batch_size = 300 #64 for linear decoder

    forecast_windows = [i for i in range(1,31)]

    for forecast_window in forecast_windows:
        
        orderbook = augment_trade_data(agg_trade, lag=0, forecast_window=forecast_window)

        features = ['price', 'lag_return',
                    'bid1', 'bidqty1', 'bid2', 'bidqty2', 'bid3', 'bidqty3', 'bid4', 'bidqty4', 'bid5', 'bidqty5',
                    'bid6', 'bidqty6', 'bid7', 'bidqty7', 'bid8', 'bidqty8', 'bid9', 'bidqty9',
                    'ask1', 'askqty1', 'ask2', 'askqty2', 'ask3', 'askqty3', 'ask4', 'askqty4', 'ask5', 'askqty5',
                    'ask6', 'askqty6', 'ask7', 'askqty7', 'ask8', 'askqty8', 'ask9', 'askqty9']

        split_index, data_x_train, data_y_train, data_x_val, data_y_val = prepare_data(np.array(orderbook[features][1_000_000-700000:1_720_000-700000]),
                                                                                                                                np.array(agg_trade.datetime[2_005_000-1200000:2_006_000-1200000]),
                                                                                                                                np.array(orderbook[features][60_000:60_600]),
                                                                                                                                np.array(agg_trade.datetime[60_000:60_600]),
                                                                                                                                config, lag=forecast_window, plot=False)


        train_loader = TimeSeriesDataset(data_x_train, data_y_train)
        val_loader = TimeSeriesDataset(data_x_val, data_y_val)
        test_loader = None

        model_custom = HFformer(n_time_series=len(features), seq_len=forecast_history, output_seq_len=1, d_model=36,
                    n_heads=6, n_layers_encoder=2, dropout=0.3, output_dim=1, forward_dim=64, use_mask=True).to(device)

        criterion = nn.MSELoss(reduction='sum')
        optimizer = optim.AdamW(model_custom.parameters(), lr=0.1, amsgrad=True)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
        warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period=1000)

        trainer(model_custom, train_loader, val_loader, test_loader, [criterion], optimizer, scheduler, warmup_scheduler, epochs, batch_size=batch_size,
            forecast_horizon=forecast_window, takes_target=False, plot_prediction=False, save_path=save_path, LAG=forecast_window)
        
        del data_x_train 
        del data_y_train
        del data_x_val
        del data_y_val

        torch.save(model_custom, f'./models_drop_duplicate_near/transformer_enclinear_forecasting_FINAL_horizon_{forecast_window}.pt')
        print(f'Done with prediction len {forecast_window}.')

## Forecast Evaluator

In [23]:
def strategy_evaluator(true, pred, weighted=False):
    '''
    Evaluates trading strategy based on correct buys and sells.
    '''
    total_buys, total_sells, total_holds = np.sum(true>0), np.sum(true<0), np.sum(true==0)
    total_correct_buys, total_correct_sells, total_correct_holds = 0, 0, 0
    for idx in range(len(true)):
        if true[idx] > 0 and pred[idx] > 0:
            total_correct_buys += 1
        elif true[idx] < 0 and pred[idx] < 0:
            total_correct_sells += 1
        elif true[idx] == 0 and pred[idx] == 0:
            total_correct_holds += 1
    total_correct_buys_r, total_correct_sells_r, total_correct_holds_r = (total_correct_buys/total_buys),(total_correct_sells/total_sells),(total_correct_holds/total_holds)
    total_correct_trades = (total_correct_buys+total_correct_sells+total_correct_holds)/(total_buys+total_sells+total_holds)
    return total_buys, total_correct_buys, total_sells, total_correct_sells, total_holds, total_correct_holds


def forecast_evaluator(test_loader, model, criterion, forecast_horizon=1, device=device, num_targets=1, save_path=None):
    '''
    Outputs evaluation metrics.
    '''
    test_data_loader = DataLoader(test_loader, batch_size=128, shuffle=False, sampler=None, batch_sampler=None, num_workers=6)
    loss, pred, true = forecast(test_data_loader, model, criterion, forecast_horizon=1, device=device, num_targets=1)
    pred, true = pred.cpu().numpy(), true.cpu().numpy()

    r2 = r2_score(true, pred)
    strategy_results = strategy_evaluator(true, pred)
    
    if save_path:
        results = {
                'model': 'Transencwithlineardec',
                'pred_len': forecast_horizon,
                'test_loss': loss,
                'r2_val_sklearn': r2,
                'correct_buys': strategy_results[1],
                'total_buys':  strategy_results[0],
                'correct_sells': strategy_results[3],
                'total_sells': strategy_results[2],
                'correct_holds': strategy_results[5],
                'total_holds': strategy_results[4],
        }
        os.makedirs(save_path_model, exist_ok=True)
      
        df.to_csv(save_path, mode='a', header=not os.path.exists(save_path), index=False)

    print(f'| test loss {loss} | b, cb, s, cs, h, ch: {strategy_results} |')

    return pred, true


## Forecasting

In [19]:
date_train = 'all' 
date_test = 'all'
drive  = None
if drive:
    agg_trade = pd.read_csv(config["paths"]["drive"]["agg_trade"]["train"]+date_train+'/orderbook.csv')    
    sys.path.append(config["paths"]["drive"]["utils"])
else:
    agg_trade = pd.read_csv(config["paths"]["local"]["agg_trade"]["train"]+date_train+'/orderbook_agg_trade_dollarvol.csv')
    agg_trade_test = pd.read_csv(config["paths"]["local"]["agg_trade"]["test"]+date_test+'/orderbook_agg_trade_dollarvol.csv')



In [21]:
agg_trade_test = agg_trade[1_200_000:]
print(agg_trade_test.shape)

(281482, 43)


In [27]:
save_path = os.path.join('/mnt/workspace/shell/models/training_details/HFTransformer/results_HFformer_tmp',
                            str(int(time.time()))+'_forecasting_results.csv')

# save_path = None

save_path_results = os.path.join('/mnt/workspace/shell/models/training_details/HFTransformer/results_HFformer_tmp',
                            str(int(time.time()))+'_list_results.pkl')

save_path_model = os.path.join('/mnt/workspace/shell/models/training_details/HFTransformer/results_HFformer_tmp',str(int(time.time()))+'_model.pth')

forecast_history = 100 
batch_size = 256 #64 for linear decoder

forecast_windows = [i for i in range(1,32)]

predictions = []

for forecast_window in forecast_windows:
    
    orderbook = augment_trade_data(agg_trade_test, lag=0, forecast_window=forecast_window)

    features = ['price', 'lag_return',
                'bid1', 'bidqty1', 'bid2', 'bidqty2', 'bid3', 'bidqty3', 'bid4', 'bidqty4', 'bid5', 'bidqty5',
                'bid6', 'bidqty6', 'bid7', 'bidqty7', 'bid8', 'bidqty8', 'bid9', 'bidqty9',
                'ask1', 'askqty1', 'ask2', 'askqty2', 'ask3', 'askqty3', 'ask4', 'askqty4', 'ask5', 'askqty5',
                'ask6', 'askqty6', 'ask7', 'askqty7', 'ask8', 'askqty8', 'ask9', 'askqty9']
    print(orderbook[features].shape)

    split_index, data_x_train, data_y_train, data_x_val, data_y_val = prepare_data(np.array(orderbook[features][:]),
                                                                                    np.array(agg_trade.datetime[2_005_000-500000:2_006_00-5000000]),
                                                                                    np.array(orderbook[features][60_000:60_600]),
                                                                                    np.array(agg_trade.datetime[60_000:60_600]),
                                                                                    config, lag=forecast_window, plot=True)

    test_loader = TimeSeriesDataset(data_x_train, data_y_train)

    model = torch.load(f'/mnt/workspace/shell/tmp/HFfMODELsept10test_tmp_1/transformer_enclinear_forecasting_FINAL_horizon_{forecast_window}.pt')
    criterion = nn.MSELoss(reduction='sum')

    pred, true = forecast_evaluator(test_loader, model, [criterion], forecast_horizon=forecast_window, device=device, num_targets=1, save_path=save_path)
    
    predictions.append((pred, true))

    print(f'Done with prediction len {forecast_window}.')

with open(save_path_results, 'wb') as f:
    pickle.dump(predictions, f)

/tmp/ipykernel_8121/2918264227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lag_return'] = np.log(df['price'].shift(forecast_window)/df['price'].shift(forecast_window+1))


(281480, 38)


/tmp/ipykernel_8121/3813266209.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'/mnt/workspace/shell/tmp/HFfMODELsept10test_tmp_1/transformer_enclin

| test loss 1332352.4474906921 | b, cb, s, cs, h, ch: (np.int64(99814), 82760, np.int64(103135), 84048, np.int64(22154), 0) |
Done with prediction len 1.
(281479, 38)


/tmp/ipykernel_8121/2918264227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lag_return'] = np.log(df['price'].shift(forecast_window)/df['price'].shift(forecast_window+1))
/tmp/ipykernel_8121/3813266209.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be a

...

| test loss 3608577.380016327 | b, cb, s, cs, h, ch: (np.int64(112420), 75424, np.int64(110133), 74819, np.int64(2505), 0) |
Done with prediction len 29.
(281451, 38)


/tmp/ipykernel_8121/2918264227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lag_return'] = np.log(df['price'].shift(forecast_window)/df['price'].shift(forecast_window+1))
/tmp/ipykernel_8121/3813266209.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be a

| test loss 3582746.993657589 | b, cb, s, cs, h, ch: (np.int64(112350), 76958, np.int64(110202), 74194, np.int64(2504), 0) |
Done with prediction len 30.
(281450, 38)


/tmp/ipykernel_8121/2918264227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lag_return'] = np.log(df['price'].shift(forecast_window)/df['price'].shift(forecast_window+1))
/tmp/ipykernel_8121/3813266209.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be a

| test loss 3602795.6707611084 | b, cb, s, cs, h, ch: (np.int64(112370), 76615, np.int64(110259), 73693, np.int64(2426), 0) |
Done with prediction len 31.
